В файле mission2ab данные по АБ-тестированию, тест был направлен на улучшение конверсии и размеров платежей пользователей. На первой странице данные по юзер айди и группе тестирования. На второй данные по платящим пользователям (юзер айди и сумма платежа). 
Задача: проанализировать тест понять какая группа лучше, стоит ли раскатывать изменения одной группы на всех пользователей. Достоверны ли полученные результаты?

Задача: проанализировать тест понять какая группа лучше, стоит ли раскатывать изменения одной группы на всех пользователей. Достоверны ли полученные результаты?

# **0. Обработка данных.**

In [ ]:
import pandas as pd
from scipy.stats import mannwhitneyu
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Сохраним первую страницу с указателем, к какой группе относится тот или иной пользователь как df0.

In [ ]:
df0=pd.read_excel('/gdrive/MyDrive/Mail test/mission2ab.xlsx', sheet_name=0)

In [ ]:
df0

,user_id,group
0,1000,A
1,1001,A
2,1002,A
3,1003,A
4,1004,A
...,...,...
19996,20996,B
19997,20997,B
19998,20998,B
19999,20999,B


Тоже самое сделаем для второй страницы с данными о платежах.

In [ ]:
df1=pd.read_excel('/gdrive/MyDrive/Mail test/mission2ab.xlsx', sheet_name=1)

In [ ]:
df1

,user_id,payment
0,8656,37
1,8233,21
2,4969,13
3,5004,33
4,5665,2
...,...,...
435,18759,13
436,16460,30
437,17064,25
438,13812,24


Смерджим обе таблицы, оставив данные только о платежной части пользователей.

In [ ]:
df_all = pd.merge(df0,df1, how='right', on='user_id')

In [ ]:
df_all

,user_id,group,payment
0,8656,A,37
1,8233,A,21
2,4969,A,13
3,5004,A,33
4,5665,A,2
...,...,...,...
435,18759,B,13
436,16460,B,30
437,17064,B,25
438,13812,B,24


# **1. Проверка основных метрик.**

# **1.1 Общая сумма платежей.**

Посчитаем общую сумму платежей, выручки для каждой из групп:

In [ ]:
df_all.pivot_table(index='group', values='payment', aggfunc='sum').reset_index()

,group,payment
0,A,5000
1,B,6600


**Вывод**

В группе B больше сумарная выручка: 6600 на фоне 5000 в группе A.

# **1.2. Средний чек для обоих групп и количество платящих пользователей.**

Создадим сводную таблицу для общей суммы платежей и для количества платежей для каждой из групп тестирования:

In [ ]:
table_temp = df_all.pivot_table(index='group', values='payment', 
                             aggfunc=['sum', 'count'])

In [ ]:
table_temp.columns = table_temp.columns.droplevel(level = 1)

Отдельно создадим средний чек за заказ для каждой из групп:

In [ ]:
table_fin = table_temp.reset_index()
table_fin['average'] = table_fin['sum'] / table_fin['count']

In [ ]:
table_fin

,group,sum,count,average
0,A,5000,200,25.0
1,B,6600,240,27.5


In [ ]:
# Example of the Mann-Whitney U Test

data1 = df_all[df_all['group'] == 'A']['payment']
data2 = df_all[df_all['group'] == 'B']['payment']
stat, p = mannwhitneyu(data1, data2)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Probably the same distribution')
else:
	print('Probably different distributions')

stat=21759.500, p=0.046
Probably different distributions


In [ ]:
px.box(df_all, x='group', y="payment")

**Вывод**

* Средний чек в группе B составляет 27.5, в то время как в группе А 25.
* Количество платежей в группе В выше и составляет 240, в группе А 200.
* Это статистически значимая разница, так как p-value получилось меньше 0.05. А это значит, что покупки в группе В выше

# **1.3. Конверсия пользователей из всех юзеров в платящих.**

Посмотрим конверсию уникальных пользователей из всех юзеров в платящих.

In [ ]:
table_visitors = df0.pivot_table(index='group', values='user_id',aggfunc='count').reset_index()

In [ ]:
table_uniq = df_all.pivot_table(index='group',values='user_id',aggfunc='nunique').reset_index()
table_uniq.columns = ['group', 'buyers']

In [ ]:
table_all = pd.merge(table_uniq,table_visitors, how='outer')

In [ ]:
table_all

,group,buyers,user_id
0,A,200,10001
1,B,240,10000


In [ ]:
table_all['conversion'] = round(table_all['buyers'] / table_all['user_id'] * 100, 2)

In [ ]:
table_all

,group,buyers,user_id,conversion
0,A,200,10001,2.0
1,B,240,10000,2.4


**Вывод**

* Конверсия в группе В составляет 2.4, в то время как в группе А 2.0
* Таким образом базовая конверсия 2%. Минимальный рост составляет 20% от 2, а значит для этого нужна выборка из 20к пользователей для каждой из групп. У нас же меньше, а значит это статистически незначимое отклонение и считать его не случайным мы не может.

https://tools.driveback.ru/sample-size.html

# **2. Общие выводы:**
* Все показатели в группе В выше:
1. Конверсия в группе В составляет 2.4, в то время как в группе А 2.0. Рост конверсии увеличился на 0.4% это не статистически значимый рост и говорить о том, что это не случайное отклонение нельзя.
2. В группе B больше сумарная выручка - 6600 на фоне 5000 в группе A.
3. Количество платежей в группе В выше и составляет 240, в группе А 200.
4. Средний чек в группе B составляет 27.5, в то время как в группе А 25. Рост среднего чека, и в целом величина покупок пользователей выше для группы B. Это статистически значимая разница, так как p-value получилось меньше 0.05. И значит это не является случайным отклонением

Решение по результатам теста

▶Так как у группы В средний чек выше, чем у группы А, даже с одинаковой конверсией пользователей можем сделать вывод, что группа В принесет больше выручки. Тест можем закончить. **Фиксируем победу группы В**
 

Минусы такого подхода.
* У нас не хватает данных по времени, из-за этого мы не можем проверить гипотезу о статистической значимости. У нас данные только по одной метрике, поэтому говорить о достоверности мы не можем.
* Размер группы тестирования мал, и не позволяет проверить статистическую значимость роста конверсии с 2% до 2.4%.

In [ ]:
table_fin['count_users'] = table_all['user_id']
table_fin

,group,sum,count,average,count_users
0,A,5000,200,25.0,10001
1,B,6600,240,27.5,10000
